In [3]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import PIL.Image


In [6]:
print( tf.keras.applications.vgg16)

<module 'keras.api._v2.keras.applications.vgg16' from 'C:\\Users\\shawn\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\keras\\api\\_v2\\keras\\applications\\vgg16\\__init__.py'>


In [7]:
def load_image(filename, max_size=None):
    image = PIL.Image.open(filename)
    
    if max_size is not None:
        factor = max_size / np.max(image.size)
        size = np.array(image.size)*factor
        size = size.astype(int)
        image = image.resize(size, PIL.Image.LANCZOS)
    return np.float32(image)

In [16]:
def save_image(image, filename):
    image = np.clip(image, 0.0, 255.0)
    image = image.astype(np.uint8)
    with open(filename, 'wb') as file:
        PIL.Image.fromarray(image).save(file, 'jpeg')
        

In [17]:
def plot_image_big(image):
    image = np.clip(image, 0.0, 255.0)
    image = image.astype(np.uint8)
    # image.show()
    plt.imshow(np.squeeze(image))  
    plt.show()
    

In [14]:
def plot_images(content_image, style_image, mixed_image):
    fig, axes = plt.subplots(1,3, figsize=(10,10))

In [18]:
random_image = load_image("images/Elevator.jpg")
plot_image_big(random_image)

TypeError: '>=' not supported between instances of 'NoneType' and 'float'